In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split


%matplotlib inline
plt.style.use('seaborn-darkgrid')
sns.set_context("talk")

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/macc-urosario/bigdataco20/main/scripts/data/red_blood_cells_csv.csv")
# df = pd.read_csv("data/red_blood_cells.csv")
df.head()

In [ ]:
df['Area_cualitativa'] = df['Area'].apply(lambda x: 'High' if x>40000 else 'Low')
df.head()

In [ ]:
df2 = df.loc[df['etiquetas']!='Drepanocits'].reset_index(drop=True)
df2['etiquetas2'] = df2.etiquetas.factorize()[0] # Esferocito es 0
df2['Area_cualitativa2'] = df2.Area_cualitativa.factorize()[0] # Low es 0

df2.etiquetas.value_counts()

Usaremos los descriptores de `MajorAxisLength` y `Mean_Green`

In [ ]:
sns.scatterplot(x='MajorAxisLength', y='Mean_Green',hue='etiquetas', data=df2,alpha=0.4);

In [ ]:
sns.boxplot(y='MajorAxisLength', x='etiquetas',data=df2);

In [ ]:
sns.boxplot(y='Mean_Green', x='etiquetas',data=df2);

## ¿Por qué no usar regresión lineal?
El modelo de regresión lineal por defecto, no es ideal y
podría hacerse mejor simplemente haciendo que todas
las probabilidades<0 sean igual a 0 y todas las
probabilidades> 1 igual a 1

In [ ]:
plt.figure(figsize=(8,5))
sns.regplot('MajorAxisLength', 'etiquetas2', data=df2)
plt.ylabel('Normal RBC')
plt.show()

# Regresión Logística

Por lo tanto, se debe encontrar una función que siempre limite la salida entre cero y uno. Una función que satiface esta condición es la función logística (también llamada sigmoid function). 

$$y = \frac{e^{\theta_0 + \theta_1X}}{1 + e^{\theta_0 + \theta_1X}}$$

Muchas veces verás la función sigmoide en un formato más simple:

$$y = \frac{1}{1 + e^{-t}}$$

Donde $t$ es solo el modelo lineal normal $t = \theta_0 + \theta_1X$. Se puede usar algo de álgebra para mostrar que las dos ecuaciones anteriores son equivalentes

Ahora se puede pensar a $y$ como la probabilidad dado un cierto valor de X ya que siempre estará entre 0 y 1. Se puede mostrar que $$log{\frac{p(X)}{1 - p(X)}} = \theta_0 + \theta_1X$$

Donde $y$ ha sido reemplazado por $p(X)$, la probabilidad de $X$. La expresión $ \frac{p(X)}{1 - p (X)} $ se conoce como *odds*. Entonces, por ejemplo, si llegaron a la final el Barcelona y el Manchester City,  estás apostando y piensas que el Barcelona ganará la champions el 80% del tiempo. El odds sería .8 / .2 = 4 o dicho de otra forma "4 a 1". Por cada 4 veces que gana el Barcelona, el Manchester City ganará una vez.

Lo que dice la regresión logística es que el *log odds* se modela mediante un modelo lineal que puede resolverse mediante regresión lineal. Esto tiene el significado literal de: dado un aumento de una unidad en una de las variables (por ejemplo, $ X_1 $), se producirá un aumento de $ \theta_1 $ en el *log odds*. O de manera equivalente, el *log odds* se multiplicará por $ e ^ {\theta_1} $.

In [ ]:
from sklearn.linear_model import LogisticRegression

X_train = df2['MajorAxisLength'].values.reshape(-1,1) 
y = df2['etiquetas2']

# Crea un array para el conjunto de test. Calcula la probabilidad
# de clasificacion y la classificacion predicha.
X_test = np.arange(df2.MajorAxisLength.min(), df2.MajorAxisLength.max()).reshape(-1,1)

clf = LogisticRegression()
clf.fit(X_train,y)
prob = clf.predict_proba(X_test)

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5))
# Left plot
sns.regplot(df2.MajorAxisLength, df2.etiquetas2, order=1, ci=None,
            scatter_kws={'color':'orange'},
            line_kws={'color':'blue', 'lw':2}, ax=ax1)
# Right plot
ax2.scatter(X_train, y, color='orange')
ax2.plot(X_test, prob[:,1], color='blue')

for ax in fig.axes:
    ax.hlines(1, xmin=ax.xaxis.get_data_interval()[0],
              xmax=ax.xaxis.get_data_interval()[1], linestyles='dashed', lw=1)
    ax.hlines(0, xmin=ax.xaxis.get_data_interval()[0],
              xmax=ax.xaxis.get_data_interval()[1], linestyles='dashed', lw=1)
    ax.set_ylabel('Probability of normal RBC')
    ax.set_xlabel('MajorAxisLength')
    ax.set_yticks([0, 0.25, 0.5, 0.75, 1.])
    ax.set_xlim(left=150)
    ax.set_ylim(top=1.1)
    ax.set_ylim(bottom=-0.1)

### Regresión logística para normal RBC usando como predictor únicamente a `MajorAxisLength`

In [ ]:
model = LogisticRegression()

X_train = df2[['MajorAxisLength']]

y = df2['etiquetas']

model.fit(X_train,y)

print("classes: {}\ncoefficientes: {}\nintercepto: {}".format(
    model.classes_,model.coef_, model.intercept_))

In [ ]:
def coefbonito(model,X):
    D = dict(coeficientes=np.c_[model.intercept_,model.coef_].ravel())
    return pd.DataFrame(D, index=['intercepto']+ X.columns.to_list())

In [ ]:
coefbonito(model,X_train)

In [ ]:
probabilidad = np.exp(-17.277293+0.074146*200)/(1+np.exp(-17.277293+0.074146*200))
print(f"La probabilidad de que el glóbulo rojo sea normal es de {probabilidad:.4f}")

In [ ]:
model.predict_proba([[200]])

In [ ]:
model.predict_proba([[250]])

### Regresión logística usando únicamente la variable cualitativa `Area_cualitativa`

In [ ]:
df2[['etiquetas','Area_cualitativa', 'MajorAxisLength', 'Mean_Green']].sample(3)

In [ ]:
model = LogisticRegression()

X_train = df2[['Area_cualitativa2']]
y = df2['etiquetas']
model.fit(X_train,y)

coefbonito(model,X_train)

In [ ]:
model.predict_proba([[1]])

In [ ]:
model.predict_proba([[0]])

### Regresión logística múltiple

In [ ]:
# El modulo "preprocessing" es muy importante en la vida real
from sklearn import preprocessing

model = LogisticRegression(C=1000)
X_train = df2[['MajorAxisLength','Area_cualitativa2','Mean_Green']]

# Se escalan los descriptores entre 0 y 1, para asegurar la convergencia.
X2 = preprocessing.scale(X_train)
y = df2['etiquetas']
# model.fit(X_train,y)
model.fit(X2,y)

coefbonito(model,X_train)

### Matriz de confusión

In [ ]:
# Funcion para calcular y visualizar la matriz de confusion 
def plotCM(ytrue, ypred, clases=None, normalize = False, ax = None):
    """ Funcion para calcular y visualizar la matriz de confusion"""
    
    if clases == None:
        clases = list(set(ytrue))
        clases.sort() # etiquetas unicas ordenadas alfabeticamente
    
    CM = confusion_matrix(ytrue,ypred, labels=clases)
    
    #Normaliza la matriz de confusion dividiendo cada fila por el total de verdaderos
    if normalize:
        CM = 100*CM / CM.sum(axis=1).reshape(-1,1) #Aprovechando el Broadcasting!
 
    df = pd.DataFrame(CM, index=clases, columns=clases)
    df.index.name = 'True'; df.columns.name = 'Predicted'
    
    sns.heatmap( df, # Visualizando la matriz de confusion
             annot=True, fmt='2.1f', cmap='ocean_r',cbar=False,square=True, annot_kws={'fontsize':16}, ax=ax )
    
    plt.show()

In [ ]:
plotCM(y, model.predict(X2))

In [ ]:
accuracy = (515+498)/(515+498+54+37)
print(f"Se obtiene una exactitud de {accuracy:.2f}")

In [ ]:
accuracy_score(y,model.predict(X2))

**¿Esta exactitud obtenida es un buen indicador del desempeño del modelo?**

No. Porque estamos evaluando el desempeño sobre el mismo conjunto con el cual fue entrenado el modelo. Si el modelo ha memorizado muy bien los datos, estaríamos en un *sobre-ajuste* (overfitting) y resultaría en una exactitud muy alta.

In [ ]:
X = df2[['MajorAxisLength','Area_cualitativa2','Mean_Green']]
X_train, X_test, y_train, y_test = train_test_split(X,df2['etiquetas'], test_size=0.35, random_state=123)

Procedemos a estandarizar los datos, pero esta vez, tenemos que tener en cuenta que se debe aplicar la misma transformación al conjunto de prueba `X_test`.

In [ ]:
scaler = preprocessing.StandardScaler() #Definimos el objeto que estandarizara
scaler.fit(X_train) # ajustamos el scaler al training set

In [ ]:
model = LogisticRegression() # Modelo
model.fit(scaler.transform(X_train), y_train) #Ajuste del modelo solo en training set

In [ ]:
y_pred = model.predict(scaler.transform(X_test))

In [ ]:
plotCM(y_test, y_pred)

In [ ]:
accuracy_score(y_test,y_pred)

### ¿Y  con las tres clases?

La regresión logística se puede a extender a más clases, realizando múltiples clasificadores binarios o modificando la función de costo para aplicarlo a múltiples clases (multinomial).

In [ ]:
model = LogisticRegression(multi_class='multinomial')
X = df[['MajorAxisLength','Area','Mean_Green']]
y = df['etiquetas']
model.fit(X,y)

y_pred = model.predict(X)

acc = accuracy_score(y,y_pred)

print(f"La exactitud de clasificación es {acc:.2f}")

In [ ]:
plotCM(y,y_pred)

# Ejercicio

¿Cómo podríamos mejorar la clasificación de los tres tipos de glóbulos rojos? 

Recuerda que hay que hacer una división training/test. Estandarizar las variables ayuda a mejorar y además, sólo hemos usado tres descriptores de los 28 que hay.

Inténtalo!

**Toma sólo las variables cuantitativas desde el `Area` hasta la `Entropy_B`**

In [ ]:
X = df.loc[:,'Area':'Entropy_B'] # Tomamos los descriptores cuantitativos originales

**Haz una división training/test con un `test_size=0.3` y una semilla `random_state=123`**

In [ ]:
# COLOCA EL CODIGO AQUI




**Crea un objeto de estandarización de las variables ajustado sobre el training set**

In [ ]:
# COLOCA EL CODIGO AQUI




**Ajusta un modelo de regresión logística sobre los datos del training set estandarizados y has una predicción sobre el test set (estandarizado).**

In [ ]:
# COLOCA EL CODIGO AQUI




**Evalúa el accuracy sobre las predicciones del test set**

In [ ]:
# COLOCA EL CODIGO AQUI




**Muestra la matriz de confusión normalizada de la predicción sobre el test set**

In [ ]:
# COLOCA EL CODIGO AQUI


